In [ ]:
from kyle_tools import rc_dict

rc_dict['legend.loc'] = 'upper right'
rc_dict['legend.fontsize'] = 11
for key,v in rc_dict.items():
    plt.rcParams[key] = v

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
from scipy.stats import sem
from scipy.linalg import expm
import numpy as np
plt.rcParams["animation.html"] = "jshtml"
from IPython.display import HTML
import matplotlib.animation as animation

%matplotlib inline

source_path = os.path.expanduser('~/source/')
sys.path.append(source_path)

In [ ]:
import kyle_tools as kt
from sus.library import ness
from sus.protocol_designer import *
from informational_states import measure

In [ ]:
from simtools.infoenginessims.api import *
from simtools.infoenginessims.analysis import running_quantities as rq
from simtools.infoenginessims.simprocedures import basic_simprocedures as sp
from simtools.infoenginessims.simprocedures import running_measurements as rp
from simtools.infoenginessims.simprocedures import trajectory_measurements as tp

In [ ]:
def periodic_truncation(positions, min=-1.5, max = 1.5):
    return (positions - min) % (max-min) + min

def three_state(positions):
    return np.round(positions)

names = ['A', 'B', 'C']
values = [-1, 0, 1]

measure_kwargs = {'transformation_function':three_state, 'outcome_names':names, 'outcome_values':values}

three_state_measurement = measure.MeasurementDevice(**measure_kwargs)

In [ ]:

k1=1
k2=.5
lmda = 1

pw_linear = ness.pwl
pwl_prot = pw_linear.trivial_protocol()

pwl_prot.params[0,:] = k1
pwl_prot.params[1,:] = k2

system = System(pwl_prot, pw_linear)



In [ ]:

k= 1
d = 1
a = 1
R= 10

lmda = 1
ness_potential = ness.three_well_periodic
prot=ness_potential.trivial_protocol()

#ness_potential = ness.three_well_periodic_mb
#prot = ness_potential.trivial_protocol()

prot.params[0,:] = d
prot.params[1,:] = k
prot.params[2,:] = a
#prot.params[3,:] = R

system = System(prot, ness_potential)


In [ ]:

system.protocol.t_f =.25
system.potential.scale=1


In [ ]:
# for paper plotting
fig,ax = plt.subplots(1,2, figsize=(1*8,1*3))
x_temp = np.linspace(-1.5,1.5, 1000)

U = ness.three_well_periodic_potential(x_temp, [d, 0, a/2 ,])

F = ness.three_well_periodic_force(x_temp, [d, 0, a ,])

ax[0].plot(x_temp, U)
ax[1].plot(x_temp, F, label='conservative force')
ax[1].plot(x_temp, np.ones(len(x_temp)), label='external drive')

xc, yc = -.33, 1.05

ax[0].scatter(xc,yc, s=82, c='r', zorder=100)
ax[0].arrow(xc, yc, .4, 0, head_width=.2, head_length=.2, color='k')
ax[0].text(xc+.3, yc+.1, '$F$')
lineargs = {'linewidth':.6, 'linestyle':'--', 'color':'k'}

ax[1].axhline(0, **lineargs)

for axis in ax:
    axis.axvline(-1.5, **lineargs)
    axis.axvline(1.5, **lineargs)
    axis.axvline(-1+.02, **lineargs)
    axis.axvline(0,**lineargs)
    axis.axvline(1-.02,**lineargs)

titles=['$U(k_B T)$','$F(k_B T/x)$']

for axis, title in zip (ax, titles):
    axis.set_ylabel(title)

for axis in ax:
    axis.set_xlabel('$x$')

fig.legend()

ax[0].axhline(2.5)
ax[0].axhline(.25)
ax[0].axhline(-.5)
ax[0].axhline(1.75)


    

In [ ]:
fig.savefig('NESS_model.pdf')

In [ ]:
%debug

In [ ]:
fig, ax = plt.subplots()
system.show_potential(1, resolution=1000, manual_domain=[[-3],[3]], ax=ax)
plt.axvline(-1.5)
plt.axvline(1.5)

offset = 1.5
plt.axhline(0+offset)
plt.axhline(3*k+offset)

In [ ]:
(k**2)/(lmda)

In [ ]:
N=100_000
init_state = system.eq_state(N, 0)

plt.hist(periodic_truncation(init_state[...,0]), bins=50);


In [ ]:

plt.hist(init_state[...,0,0], bins=30, alpha=.5, density=True);
plt.hist(init_state[...,0,1], bins=30, alpha=.5, density=True);

In [ ]:
'''
N = 3_000
init_state = system.eq_state(3*N,t=0, manual_domain=[[-.5],[.5]])
init_state[:N,...,0] += 1
init_state[N:2*N,...,0] -= 1
init_state[...,0] = periodic_truncation(init_state[...,0])

np.random.shuffle(init_state)

plt.hist(init_state[...,0], bins=60);
'''

In [ ]:
step_skip = 5
#step_skip = 50

procs = [
        sp.ReturnFinalState(),
        sp.MeasureAllState(trial_request=np.s_[:],step_request=np.s_[::step_skip]),
        rp.MeasureFinalValue(rp.get_dE, 'final_E'), 
        rp.MeasureAllValue(rp.get_dE, 'all_E'),
        sp.MeasureMeanValue(rp.get_current_state, 'means')]

In [ ]:
len(loaded_data)

In [ ]:
#files = [f'./data/jinghao_mystery_traj_dt_p001_ep_p25_{n}.npy' for n in [f'0{N}' for N in range(5,9)]]
#loaded_data = 
loaded_data = np.load('./data/jinghao_mystery_traj_dt_p001_ep_p25_12.npy')
N=600_000
#indices = [ int(i) for i in np.linspace(0,200,int(N/len(loaded_data))) ]
init_state = np.empty( (N,1,2) )
#init_state[:] = np.vstack( [loaded_data[:,i,...] for i in indices])
init_state = loaded_data[:,-1,None,:]

In [ ]:
loaded_data.shape

In [ ]:
loaded_data = [np.load(i) for i in files]
all_load = np.vstack(loaded_data)


In [ ]:
all_load.shape

In [ ]:
np.log(24_000_000)

In [ ]:
from kyle_tools import rc_dict

for k,v in rc_dict.items():
    plt.rcParams[k] = v

ld = all_load[:,::2,...].reshape(-1, 1,2)
fig, ax = plt.subplots(figsize=(5,3))
#ax[0].hist2d(init_state[...,0,0],init_state[...,0,1], bins=100 );
ax.hist2d(ld[...,0,0],ld[...,0,1], bins=500, cmap='inferno');

ax.set_ylim(-4,4)
ax.set_xlabel('$x$')
ax.set_ylabel('$p$')



In [ ]:
fig.savefig('cyclic_ness_phase_space_5.png')

In [ ]:
#init_state=final_state
from quick_sim import setup_sim
#dt = 1/1000
dt = 1/5000


sim = setup_sim(system, init_state, damping=lmda, temp=1, dt=dt, procedures=procs)

In [ ]:
%%time
sim.output = sim.run(verbose=True)

In [ ]:
all_state = sim.output.all_state['states'].squeeze()
p_all_state = all_state.copy()
p_all_state[...,0] = periodic_truncation(p_all_state[...,0])
inf_all_state = three_state(p_all_state[...,0])
all_E = sim.output.all_E
final_E = sim.output.final_E
final_state = sim.output.final_state
p_final_state = periodic_truncation(final_state[...,0])
t= np.linspace(0, system.protocol.t_f, 1+int(system.protocol.t_f/dt))
all_means = sim.output.means['values']


In [ ]:
all_state.shape

In [ ]:
#is_dict = kt.separate_by_state(init_state[:,0,1,0], **measure_kwargs)
#fs_dict = kt.separate_by_state(p_final_state[:,0,1,0], **measure_kwargs)

In [ ]:
#plotting the trajectories along a particular axis
end_plot_time = 1*system.protocol.t_f #* 1 / 100
trial_indices = np.s_[:1000]

rq.plot_running_quantity(all_state[trial_indices,:,0,0],
                                                  final_time=end_plot_time,
                                                  end_plot_time=end_plot_time, title='x v t', alpha=.05)


rq.plot_running_quantity(p_all_state[trial_indices,:,0,0],
                                                  final_time=end_plot_time,
                                                  end_plot_time=end_plot_time, title='x v t', alpha=.05)


In [ ]:
%%capture
ani, fig, ax = kt.animate_hist_1D(p_all_state, 40, frame_skip=10, lims=[-1.5,1.5])
HTML(ani.to_jshtml(fps=24))

In [ ]:
ani

In [ ]:
fig, ax= plt.subplots(1,4, figsize=(20,5),sharex=True, sharey=True)
t_vals = np.linspace(0, len(p_all_state[0,:,0,0])-1, 4)
for i, item in enumerate(t_vals):
    ax[i].hist(p_all_state[:,int(item),0,0], bins=60)

In [ ]:
fig, ax= plt.subplots(1,4, figsize=(20,5), sharex=True, sharey=True)
t_vals = np.linspace(0, len(p_all_state[0,:,0,0])-1, 4)
for i, item in enumerate(t_vals):
    ax[i].hist(p_all_state[:,int(item),0,1], bins=60, density=True)

In [ ]:
%%capture

ani,_,_ = kt.animate_sim(p_all_state[:100,:,0,:], frame_skip=1)
HTML(ani.to_jshtml(fps=30))

In [ ]:
ani

In [ ]:
#ani.save('jinghao_qualitative')

In [ ]:
#indx = [1296, 40, 2900] 
test = ( np.max( np.abs(all_state[:,:,:,0]), axis=1 ) > 2)[:,0]
rq.plot_running_quantity(all_state[test,:,0,0],
                                                  final_time=end_plot_time,
                                                  end_plot_time=end_plot_time, title='x v t rare', alpha=.1)

In [ ]:
p_all_state.shape
print(1/500)

In [ ]:
dt*step_skip

In [ ]:
#np.save('PWL_UD_NESS_dt_p005_ep_p55_01', p_all_state, allow_pickle=False)
np.save('./data/jinghao_mystery_traj_dt_p001_ep_p25_14', p_all_state, allow_pickle=False)

In [ ]:
#p_all_state = np.load('./PWL_UD_NESS_dt_p005_ep_p55_01.npy', allow_pickle=False)

In [ ]:
fig, ax = plt.subplots()
ax.hist(np.diff(p_all_state[:,...,0]));
ax.set_yscale('log')

In [ ]:
files = [f'./data/jinghao_mystery_traj_dt_p001_ep_p25_{n}.npy' for n in [f'{N:02}' for N in range(5,8)]]

process = ([np.array(np.load(item)).squeeze()[:,:201,:] for item in files])
process = np.vstack(process)

d=3

s =  process.shape

new_len = int(s[1]/d)
new_process = np.empty( (d*s[0], new_len, s[-1]) )
for i in range(d):
    new_process[i*s[0]:(i+1)*s[0],...] = process[:,i*new_len:(i+1)*new_len ,...]
    
#new_process = new_process.reshape(d*s[0]*s[1], new_len,s[-1])

print(process.shape, new_process.shape)
process = new_process

In [ ]:
process.shape

In [ ]:
pas = np.vstack( [p_all_state[:,:201,...].squeeze(), process] )

In [ ]:
#p_all_state = process

p_jumps = np.diff(p_all_state[...,0]) > 2
n_jumps = np.diff(p_all_state[...,0]) < -2
idx = 12
plt.plot(n_jumps[idx,:], linestyle='none', marker='o');
plt.plot(p_all_state[idx,:,0].T);

In [ ]:
dt=1/5000
step_skip=5

cg = 1
K = 1

p_all_state = process
dx_state = p_all_state[:,::cg,0]
#dx_state = all_state[:,::cg,0]
cyclic_lims = [-1.5,1.5]
#cyclic_lims = None


dx = np.diff(dx_state, axis=1)



if cyclic_lims is not None:
        domain = np.diff(np.asarray(cyclic_lims))
        jump_back = dx < -.8*domain
        jump_forw = dx > .8*domain
        dx[jump_back] = dx[jump_back] + domain
        dx[jump_forw] = dx[jump_forw] - domain

ast = K*dx
ast_d = ast/(cg*dt*step_skip)
ent = ast_d.mean(axis=0)
err = ast_d.std(axis=0)

fig, ax  = plt.subplots()

ax.errorbar( np.linspace(0,1, len(ast_d[0,:])), ent, yerr=3*err/np.sqrt(len(dx)), linestyle='none', marker='D' );
ax.axhline(ent.mean())
print(ent.mean())

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,5))
ax[0].plot(p_jumps.sum(axis=0))
ax[0].plot(n_jumps.sum(axis=0))

ent_j = 3*(K)*(n_jumps.mean(axis=0)-p_jumps.mean(axis=0))/(dt*step_skip)
#ent_j = 3*k*(n_jumps.mean(axis=0)-p_jumps.mean(axis=0))/(dt*step_skip)


ax[1].plot( ent_j)
ax[2].hist( ent_j,bins=np.linspace(.17,.34,int(3*np.log(len(n_jumps)))))
ax[1].axhline( ent_j.mean())
ax[1].axhline( ent_j.mean()+3*ent_j.std())
ax[1].axhline( ent_j.mean()-3*ent_j.std())
print(ent_j.mean())



In [ ]:
#ani.save('wind_tunnels.mp4')

In [ ]:
plt.plot(sigma_mean)

In [ ]:
plt.scatter(t[::10],inf_all_state[1,::10,...,0], s=2)
#plt.scatter(t[::5],inf_all_state[5,::5,...,0])
#plt.scatter(t[::5],inf_all_state[10,::5,...,0])


In [ ]:
def state_prob(x, mu_x, sigma_x):
    x, v = x[...,0,0], x[...,0,1]

    pdf_v = np.exp(-(v-k/lmda)**2)
    pdf_x = np.multiply(1/sigma_x,np.exp(-.5*((x-mu_x)/sigma_x)**2))
    
    return 1/(2*np.pi)*pdf_v*pdf_x

def scaled_var(array):
    return np.var(array, axis=0)/(np.mean(array,axis=0)**2)

In [ ]:
def FT_avg(array, sigma=None, axis=0):
    if sigma is None:
        sigma=array
    array = array * (1-np.exp(-sigma))
    return np.mean(array, axis=axis)

In [ ]:
std_v = np.std(all_state[...,0,1], axis=0)
plt.plot(t,std_v)
plt.axhline(1)

In [ ]:
std = np.std(all_state[...,0,0], axis=0)
mean = np.mean(all_state[...,0,0], axis=0)[1:]

plt.plot(t, np.divide(std,np.sqrt(init_var+2*t/lmda)) )
plt.title('x_std divided by analytic form')


In [ ]:
all_prob = state_prob(all_state, t*k/lmda, np.sqrt(init_var+2*t/lmda))
init_prob = state_prob(init_state, 0, np.sqrt(init_var))

In [ ]:
for item in -np.log(np.divide(np.transpose(all_prob[:5]),init_prob[:5])).transpose():
    plt.plot(t[1:],item);

In [ ]:
dH = -np.log(np.divide(np.transpose(all_prob),init_prob)).transpose()

In [ ]:
fig, ax = plt.subplots()

#plt.plot(np.mean(dH, axis=0))
plt.plot(np.mean(all_E, axis=0))



ax.set_xlabel('$\\nu$ hi')


In [ ]:
sigma_dist = -(all_E + dH)
sgma_mean = np.mean(sigma_dist, axis=0)
sgma_std = np.std(sigma_dist, axis=0)

fig,ax=plt.subplots(figsize=(10,8))

ax.errorbar(t, sigma_dist.mean(axis=0), yerr=sgma_std, alpha=.2, c='b')
ax.errorbar(t, -k*(all_state[...,0].mean(axis=0)[:,0]), yerr=k*np.sqrt(init_var+2*t/lmda), alpha=.1, c='g')



ax.legend(['$\sigma(t) = H(x_t)-H(x_0) - ln(Pr(x_t)/Pr(x_0))$', '$\sigma(t)=k * x_t$' ])
ax.set_title('$\\langle \sigma \\rangle \pm s_{\sigma}$ with reference frame moving at $v= \\nu t$')
ax.set_xlabel('t')
ax.set_ylabel('$\\langle \sigma \\rangle - \\nu t$')



In [ ]:
plt.plot(sgma_std)
plt.plot(np.sqrt(2*t/lmda))


In [ ]:
plt.errorbar(t, -current.mean(axis=0), current.std(axis=0) )

In [ ]:
min_bound[:10]

In [ ]:
plt.plot(info_curr.mean(axis=0)[:300]**2)
plt.plot(info_curr.var(axis=0)[:300])

In [ ]:
plt.plot(np.var(info_curr, axis=0));

In [ ]:
current = ((all_state[...,0,0].transpose() - init_state[...,0,0]).transpose())
epsilon = scaled_var(current)

i_epsilon = scaled_var(info_curr)



In [ ]:
np.shape(i_epsilon)

In [ ]:
sgma_mean[100]

In [ ]:
from kyle_tools.utilities import inv_xtanhx
inv = []
for i,item in enumerate(sgma_mean):
    
    print('\r {}of{}'.format(i,len(sgma_mean)),end='')
    inv.append(inv_xtanhx(item/2))

In [ ]:
new_inv = np.zeros(len(inv))
for i, item in enumerate(inv):
    new_inv[i] = item

In [ ]:
hasegawa = 2/(np.exp(sgma_mean)-1)
tur_de_force = 1/(np.sinh(new_inv))**2

In [ ]:
min_bound = 1/np.mean(np.tanh(sigma_dist/2),axis=0) -1
analytic_min = 2*np.exp(-sgma_mean/2)

In [ ]:
plt.plot(min_bound[100:])

In [ ]:
idx=np.s_[:]
tn=t[:]
fig, ax = plt.subplots(figsize=(15,12))

for item in [min_bound/(2/sgma_mean), min_bound/hasegawa, min_bound/tur_de_force, min_bound/epsilon, min_bound/i_epsilon]:
    ax.semilogy(tn[idx], item[idx])

ax.axhline(1,c='k', linestyle='--')
plt.legend(['HG','HVV','TGGL', '$\\epsilon^2_{J_dx}$','$\\epsilon^2_{J_dS}$'])

In [ ]:
x_means = all_means[...,0]
x_std = x_means[:,1] * np.sqrt(3*N)
theory = np.sqrt(2*t/lmda+x_std[0]**2)
plt.plot(t, x_std)
plt.plot(t, theory, c='k')

In [ ]:
plt.errorbar(t, all_state[...,0,0].mean(axis=0), yerr=all_state[...,0,0].std(axis=0))
plt.plot(t, t*k/lmda, c='k')

In [ ]:
dE = np.diff(all_E,axis=1).mean(axis=0)
dE_var = np.var(-dE)
dE_mean = np.mean(-dE)
plt.plot(dE)

In [ ]:
dH = .5*np.log(x_std[0]/x_std)

In [ ]:
bound1 = 2*k**2/lmda**2
bound2 = k**2/lmda
plt.errorbar(t,(-all_E.mean(axis=0)+dH[:,0])/t-bound2, yerr=-sem(all_E)/t)
#plt.axhline(bound1, c='k')
print(bound1)

In [ ]:
dE_mean

In [ ]:
plt.hist(-steps*dE_mean, bins=40);
print(np.mean(-steps*dE_mean)/system.protocol.t_f)



In [ ]:
plt.hist(-all_E[:,int(steps/2)], bins=40);
mn = -final_E.mean()/system.protocol.t_f
std = np.var(-final_E/steps)
print(mn)

In [ ]:
plt.hist(-final_E, bins=40);
mn = -final_E.mean()/system.protocol.t_f
std = np.var(-final_E/steps)
print(mn, 2*(2/1.5)**2)

In [ ]:
short_procs = [
        sp.ReturnFinalState(),
        rp.MeasureFinalValue(rp.get_dE, 'final_E'), 
        ]

final_E = [[],[]]
final_S = []
initial_S = []
bound=[]

lda_list = np.linspace(3,10, 20)
for item in lda_list:
        lmda=item
        
        tau=1
        k=5

        prot.params[1,:] = k
        system = System(prot, ness_potential)
        system.protocol.t_f =tau

        N=25_000
        init_state = np.random.normal(0, np.sqrt(2*10), (N,1,2))
        init_state[...,1] = np.random.normal(k/lmda, 1,(N,1))
        

        sim = setup_sim(system, init_state, damping=lmda, temp=1, nsteps=int(system.protocol.t_f/dt), procedures=procs)
        sim.output=sim.run(verbose=True)
        print(':',lmda)
        

        final_E[0].append(-sim.output.final_E.mean()/tau)
        final_E[1].append(sem(sim.output.final_E)/tau)
        final_S.append(sim.output.final_state)
        initial_S.append(init_state)
        bound.append(2*k**2/(lmda**2))

        

        
        



In [ ]:
k**2 / lda_list

In [ ]:
std

In [ ]:
final_dH = []
for i,item in enumerate(final_E[0]):
    final_dH.append(item+ dH[i])

In [ ]:
indices=np.s_[:]
plt.semilogy(lda_list[indices], final_dH[indices], marker='.')
plt.semilogy(lda_list[indices], bound[indices], c='k', marker='+')
plt.semilogy(lda_list[indices], k**2 / lda_list[indices], c='r', linestyle='--')
plt.legend(['$\sigma$','bound','analytic $\Sigma^{bath}$'])
plt.xlabel('$\lambda$')

In [ ]:
dH=[]
dH_theory=[]
std=[]
std_init = []
fs_list = list(zip(final_S, initial_S))
nrml_test = [[],[]]

for i,item in enumerate(fs_list):

    std_f, skew_f, kurt_f = item[0][...,0].std(), skew(item[0][...,0]), kurtosis(item[0][...,0])
    std_i = item[1][...,0].std()
    nrml_test[0].append(skew_f)
    nrml_test[1].append(kurt_f)
    std.append(std_f)
    std_init.append(std_i)
    
    dH.append((np.log(std_f/std_i)/tau))

    dH_theory.append(.5*np.log(1+(2*tau/(20*lda_list[i])))/tau)

In [ ]:
plt.plot(dH)
plt.plot(dH_theory, c='k')

In [ ]:
dt_measurement = measure.Measurement(p_all_state[:,:,:,0], three_state_measurement)
dt_ensemble = dt_measurement.calculate_trajectory_statistics()
#dt_dyn = dt_ensemble.calculate_dynamics()

In [ ]:
np.max(dt_ensemble.unique_traj)

In [ ]:
plt.hist(info_weights)

In [ ]:
info_curr, info_weights = find_current(dt_ensemble)

In [ ]:
def find_current(traj_ensemble):
    trajectories = traj_ensemble.unique_traj
    probs = traj_ensemble.traj_probs

    current = trajectories[...,:-1]
    next = trajectories[...,1:]

    transition_trajectories = transition_value(current, next)

    return np.cumsum(transition_trajectories, axis=1), probs


def transition_value(old, new):
    delta = new-old
    delta[delta==2] = -1
    delta[delta==-2] = 1
    N, steps = np.shape(delta)
    transition_trajectories = np.zeros((N, steps+1))
    transition_trajectories[:,1:] = delta
    return transition_trajectories
    




In [ ]:
dt_ensemble.traj_probs[:10]

In [ ]:
dt_measurement = measure.Measurement(p_all_state[:,::2,:,0], three_state_measurement)
dt_ensemble = dt_measurement.calculate_trajectory_statistics()
dt_dyn = dt_ensemble.calculate_dynamics()

dt_means = np.mean(dt_dyn, axis=0)
for i in range(3):
    dt_means[i,i] -= 1
dt_stds = np.std(dyn, axis=0)


Q_dt = dt_means/(dt)

P_dt_1 = expm(Q_dt)

In [ ]:
from scipy.linalg import eig
eig(Q_dt, left=True)

In [ ]:
np.matmul((1/3,1/3,1/3), Q_dt)

In [ ]:
np.matmul((1/3,1/3,1/3), P_dt.transpose() )

In [ ]:
scipy.matmual(Q_dt,(1/3,1/3,1/3))

In [ ]:
dt_means

In [ ]:
P_dt_1

In [ ]:

def get_dQ(n):
    dict = {'Q':[],'P(1)':[]}
    dict['n'] = n
    for item in list(n):
        
        measurement = measure.Measurement(p_all_state[:,::item,:,0], three_state_measurement)
        ensemble = measurement.calculate_trajectory_statistics()
        dyn = ensemble.calculate_dynamics()
        means = np.mean(dyn, axis=0)
        for i in range(3):
            means[i,i] -= 1
        Q_dt = dt_means/(item*dt)
        P_dt = expm(Q_dt)
        dict['Q'].append(Q_dt)
        dict['P(1)'].append(P_dt)
    return dict

In [ ]:
ctmc_list = get_dQ([64,32,16, 8, 4, 2, 1])

In [ ]:
measurement = measure.Measurement(p_all_state[:,::10,:,0], three_state_measurement)
ensemble = measurement.calculate_trajectory_statistics()

In [ ]:
ctmc_list['Q'][0]

In [ ]:
ctmc_a = np.asarray(ctmc_list['Q'])

#plt.plot(ctmc_list['n'], ctmc_a[:,0,0])
#plt.plot(ctmc_list['n'], ctmc_a[:,1,1])
#plt.plot(ctmc_list['n'], ctmc_a[:,2,2])
plt.plot(dt*np.asarray(ctmc_list['n']), ctmc_a[:,0,0], marker='o')

In [ ]:
n1=[1,2,3]
list(n1)

In [ ]:
means = np.mean(dyn, axis=0)
for i in range(3):
    means[i,i] -= 1
stds = np.std(dyn, axis=0)

In [ ]:
for item in [64, 32, 16, 8, 4, 2]:

In [ ]:
P_dt_1

In [ ]:
P_10dt_1

In [ ]:
Q_dt

In [ ]:
P_200dt_1

In [ ]:
np.linalg.det(P)

In [ ]:
np.linalg.matrix_power(P,10)

In [ ]:
v_i_2 = (1-means[0,0])/(10*dt)
v_i_2

In [ ]:
v_i = (1-means[0,0])/dt
v_i

In [ ]:
means[0,1]/(v_i*dt)

In [ ]:
means[0,2]/(v_i*10*dt)

In [ ]:
means[0,2]/(v_i*dt)

In [ ]:
means[0,1]/(v_i*dt)/v_i

In [ ]:
means[0,2]/(v_i*dt)/v_i

In [ ]:
ensemble.markovity_test(dyn);

In [ ]:
np.mean(dyn, axis=0)[1:,0]/(np.mean(dyn, axis=0)[1:,0].sum())

In [ ]:
fig, ax = plt.subplots()
ax.set_title('$\\langle W \\rangle_{min}$')